In [1]:
# setup and import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep

In [2]:
# get chromedriver executable path
chmdriver=!which chromedriver

In [3]:
# open browser
executable_path={"executable_path":chmdriver[0]}
browser=Browser("chrome",**executable_path,headless=False)

## NASA Mars News
* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text

In [4]:
# connect to url
url="https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
# use BeautifulSoup to get latest news title and paragraph text
html=browser.html
soup=bs(html,"html.parser")
result=soup.find("li",class_="slide")
for r in result:
    news_title=r.find("div",class_="content_title").text.strip()
    news_p=r.find("div",class_="rollover_description_inner").text.strip()
    print(f'{news_title}\n-----\n{news_p}')

NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
-----
Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.


## JPL Mars Space Images - Featured Image
* Get url of featured image on [JPL](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)

In [6]:
# connect to url
url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [7]:
# use BeautifulSoup to get featured image
html=browser.html
soup=bs(html,"html.parser")
base_url=soup.find("a",id="jpl_logo")["href"]
img_url=soup.find("a",id="full_image")["data-fancybox-href"]
featured_image_url="https:"+base_url+img_url[1:]
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19168_ip.jpg


## Mars Facts
* Scrape the [Mars Facts](https://space-facts.com/mars/) webpage table for facts about the planet

In [8]:
# set url
url="https://space-facts.com/mars/"

In [9]:
# get table
tables=pd.read_html(url)
table=tables[0]

In [10]:
table.columns=['Description','Mars']
table.set_index('Description',inplace=True)
table

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# convert pandas dataframe to html table
html_table=table.to_html()
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres
* Visit the [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) for high resolution images of Mars' hemispheres

In [12]:
# connect to url
url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [13]:
# get hemisphere title and link to each hemisphere
title=[]
hemi_url=[]

html=browser.html
soup = bs(html,"html.parser")
hemi=soup.find_all("div",class_="description")
for h in hemi:
    title.append(h.a.h3.text)
    hemi_url.append("https://astrogeology.usgs.gov"+h.a["href"])

In [14]:
# visit hemisphere url to obtaine image url
img_url=[]

for url in hemi_url:
    browser.visit(url)
    html=browser.html
    soup = bs(html,'html.parser')
    for link in soup.find_all("a"):
        if link.text=="Sample":
            img_url.append(link["href"])

In [15]:
# create a list containing dictionary for each hemisphere
hemisphere_image_urls=[]
for i in range(4):
    title_short=title[i].replace(" Enhanced","") # clean up title
    hemisphere_image_urls.append({"title":title_short,"img_url":img_url[i]})
    print(hemisphere_image_urls[i])

{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


In [16]:
browser.quit()